# Compiling RCS to Clifford+RZ (With Qrack)

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
width = 36
depth = 6
max_magic = 6
shots = 128

In [2]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

import os
import collections
import math
import random

from pyqrack import QrackSimulator, Pauli
from mitiq import zne
from qiskit.circuit.quantumcircuit import QuantumCircuit

# Change these according to your OpenCL devices and system specifications:
os.environ['QRACK_MAX_PAGE_QB']='27'
os.environ['QRACK_MAX_ALLOC_MB']='15872'
os.environ['QRACK_MAX_PAGING_QB']='30'
os.environ['QRACK_MAX_CPU_QB']='30'
# os.environ['QRACK_QUNITMULTI_DEVICES']='1'
# os.environ['QRACK_QPAGER_DEVICES']='1'

In [3]:
def x_to_y(circ, q):
    circ.s(q)

def x_to_z(circ, q):
    circ.h(q)

def y_to_z(circ, q):
    circ.s(q)
    circ.h(q)   

def y_to_x(circ, q):
    circ.sdg(q)

def z_to_x(circ, q):
    circ.h(q)

def z_to_y(circ, q):
    circ.h(q)
    circ.s(q)

def cx(circ, q1, q2):
    circ.cx(q1, q2)

def cy(circ, q1, q2):
    circ.cy(q1, q2)

def cz(circ, q1, q2):
    circ.cz(q1, q2)

def acx(circ, q1, q2):
    circ.x(q1)
    circ.cx(q1, q2)
    circ.x(q1)

def acy(circ, q1, q2):
    circ.x(q1)
    circ.cy(q1, q2)

def acz(circ, q1, q2):
    circ.x(q1)
    circ.cz(q1, q2)
    circ.x(q1)

def swap(circ, q1, q2):
    circ.swap(q1, q2)

def nswap(circ, q1, q2):
    circ.x(q1)
    circ.cz(q1, q2)
    circ.x(q1)
    circ.swap(q1, q2)
    circ.x(q1)
    circ.cz(q1, q2)
    circ.x(q1)

def pswap(circ, q1, q2):
    circ.x(q1)
    circ.cz(q1, q2)
    circ.x(q1)
    circ.swap(q1, q2)   

def mswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.x(q1)
    circ.cz(q1, q2)
    circ.x(q1)

def iswap(circ, q1, q2):
    circ.iswap(q1, q2)

def iiswap(circ, q1, q2):
    circ.iswap(q1, q2)
    circ.iswap(q1, q2)
    circ.iswap(q1, q2)

def random_circuit(width, depth, max_magic):
    t_count = 0
    
    single_bit_gates = { 0: (z_to_x, z_to_y), 1: (x_to_y, x_to_z), 2: (y_to_z, y_to_x) }
    single_bit_mirrors = { 0: (x_to_z, y_to_z), 1: (y_to_x, z_to_x), 2: (z_to_y, x_to_y) } 
    two_bit_gates = swap, pswap, mswap, nswap, iswap, iiswap, cx, cy, cz, acx, acy, acz
    two_bit_mirrors = swap, mswap, pswap, nswap, iiswap, iswap, cx, cy, cz, acx, acy, acz
    
    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    row_len = math.ceil(math.sqrt(width))

    # Don't repeat bases:
    bases = [0] * width
    directions = [0] * width

    circuit = QuantumCircuit(width, width)
    
    for i in range(depth):
        # Single bit gates
        single_qubit_layer = []
        single_qubit_mirror = []
        for j in range(width):
            # Reset basis, every third layer
            if i % 3 == 0:
                bases[j] = random.randint(0, 2)
                directions[j] = random.randint(0, 1)
            
            # Sequential basis switch
            gate = single_bit_gates[bases[j]][directions[j]]
            gate(circuit, j)

            # Cycle through all 3 Pauli axes, every 3 layers
            if directions[j]:
                bases[j] -= 1
                if bases[j] < 0:
                    bases[j] += 3
            else:
                bases[j] += 1
                if bases[j] > 2:
                    bases[j] -= 3

            # Rotate around local Z axis
            rnd = random.randint(0, 3)
            if rnd == 0:
                circuit.s(j)
            elif rnd == 1:
                circuit.z(j)
            elif rnd == 2:
                circuit.sdg(j)
            
            if (t_count < max_magic) and (3 * width * width * random.random() / max_magic) < 1:
                angle = random.uniform(0, 2 * math.pi)
                circuit.rz(angle, j)
                t_count += 1

        # Nearest-neighbor couplers:
        ############################
        gate = gateSequence.pop(0)
        gateSequence.append(gate)
        for row in range(1, row_len, 2):
            for col in range(row_len):
                temp_row = row
                temp_col = col
                temp_row = temp_row + (1 if (gate & 2) else -1);
                temp_col = temp_col + (1 if (gate & 1) else 0)

                if (temp_row < 0) or (temp_col < 0) or (temp_row >= row_len) or (temp_col >= row_len):
                    continue

                b1 = row * row_len + col
                b2 = temp_row * row_len + temp_col

                if (b1 >= width) or (b2 >= width):
                    continue

                g = random.choice(two_bit_gates)
                g(circuit, b1, b2)
    
    return circuit

Run these next two cells several times to produce a noisy value, (optionally tuning the width, depth, and magic settings above,) and run the third cell to mitigate with Mitiq.

In [4]:
circ = random_circuit(width, depth, max_magic)

In [5]:
qubits = list(range(width))

def execute(circuit, qsim):
    """Returns the mirror circuit expectation value for unsigned integer overall bit string."""

    circ = circuit.compose(circuit.inverse())

    qsim.reset_all()
    qsim.run_qiskit_circuit(circ, 0)

    return sum(qsim.measure_shots(qubits, shots)) / shots

qsim = QrackSimulator(width)
qsim.set_weak_sampling(True)

noisy_value = execute(circ, qsim)
print("Potentially noisy value: ", noisy_value)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Potentially noisy value:  0.0


In [6]:
if noisy_value > 1e-6:
    print("Error with ZNE: ", zne.execute_with_zne(circ, execute))
else:
    print("No mitigation required. (Ideal value.)")

No mitigation required. (Ideal value.)
